In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
import mediapipe as mp

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import feedforward

## Setup folder and register the training data

In [ ]:
DATA_PATH = os.path.join('dist') 

In [ ]:
feedforward.create_folder_if_not_exists(DATA_PATH)

In [ ]:
# feedforward.clear_folder(DATA_PATH)

In [ ]:
num_videos = 20
num_frames_per_video = 100
prefix = 'arrow'

In [ ]:
feedforward.register_videos(DATA_PATH, prefix, num_videos, num_frames_per_video)

## Classification

### Data Augmentation

In [ ]:
prefix_for_augmented = "augmented_"
feedforward.augment_dataset(DATA_PATH, prefix_for_augmented)

In [ ]:
feedforward.delete_files_with_prefix(DATA_PATH, prefix_for_augmented)

### Building the training set

In [ ]:
WINDOW_LEN = 20
TERGET_LANDMARK = mp.solutions.holistic.PoseLandmark.RIGHT_WRIST
X_training, y_training = feedforward.build_training_data(DATA_PATH, WINDOW_LEN, TERGET_LANDMARK)
X_training.shape, y_training.shape

### Build and Train the model

In [ ]:
model = Sequential()

model.add(LSTM(64, input_shape=(X_training.shape[1], X_training.shape[2]), activation='relu', return_sequences=True)) # LSTM layer with input shape (X_training.shape[1], X_training.shape[2]))
model.add(LSTM(64, activation='relu', return_sequences=False)) # LSTM layer with return_sequences=False for the final prediction# LSTM layer with return_sequences=False for the final prediction
model.add(Dense(2))# Dense output layer with 2 units

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

history = model.fit(X_training, y_training, batch_size=32, epochs=100, validation_split=0.2)

model.summary()

## Test

In [ ]:
cap = cv2.VideoCapture(feedforward.CAMERA_INDEX)
holistic = mp.solutions.holistic.Holistic()
frame_buffer = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = holistic.process(frame_rgb)

    if results.pose_landmarks:
        target_coords = results.pose_landmarks.landmark[TERGET_LANDMARK]

        height, width, _ = frame.shape

        pixel_coords_x, pixel_coords_y = int(target_coords.x * width), int(target_coords.y * height)
        cv2.circle(frame, (pixel_coords_x, pixel_coords_y), 5, (255, 0, 0), -1)

        frame_buffer.append([
            target_coords.x,
            target_coords.y,
            # target_coords.z
        ])

        if len(frame_buffer) >= WINDOW_LEN/2:
            prediction_input = np.array(frame_buffer)[np.newaxis, :, :]
            prediction = model.predict(prediction_input)

            prediction_x = int(prediction[0][0] * width)
            prediction_y = int(prediction[0][1] * height)
            
            cv2.arrowedLine(frame, (pixel_coords_x, pixel_coords_y), (prediction_x, prediction_y), (255, 0, 255), 2)

            frame_buffer = frame_buffer[1:]

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()